In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import ipywidgets as ipy

In [ ]:

@ipy.interact(i=ipy.widgets.IntSlider())
def f(i):
    img_m = plt.imread('/mnt/ml-team/dsb_2018/kuba/debug/proba_map{}.png'.format(i))
    img_b = plt.imread('/mnt/ml-team/dsb_2018/kuba/debug/binary_map{}.png'.format(i))
    plt.subplot(1,2,1)
    plt.imshow(img_m)
    plt.subplot(1,2,2)
    plt.imshow(img_b)
    plt.show()

In [1]:
%load_ext autoreload
%autoreload 2

from steps.pytorch.architectures.unet import UNet


params = {'n_filters': 32,
  'conv_kernel': 3,
  'pool_kernel': 2,
  'pool_stride': 1,
  'repeat_blocks': 3,
  'batch_norm': 0, # 0 or 1
          
  'dropout': 0.1,  # interval (0.0; 1.0)
  'num_classes': 2,
  'in_channels': 3}
unet = UNet(**params)


32
64
128
64
128
256


In [2]:
unet.down_convs

ModuleList (
  (0): DownConv (
    (down_conv): Sequential (
      (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU ()
      (2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
      (3): ReLU ()
      (4): Dropout (p = 0.1)
    )
  )
  (1): DownConv (
    (down_conv): Sequential (
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU ()
      (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (3): ReLU ()
      (4): Dropout (p = 0.1)
    )
  )
  (2): DownConv (
    (down_conv): Sequential (
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU ()
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (3): ReLU ()
      (4): Dropout (p = 0.1)
    )
  )
)